En este script preprocesamos el dataset de titulares clickbait final de ObtencionTitularesClickbait. Primero, mejoramos el dataset final obtenido con el scraper de X para quedarnos con las columnas que nos interesan. Además, obtenemos las urls reales de las noticias, no las de los posts en las que aparecen.

In [ ]:
import pandas as pd
import re

# Cargar el archivo original
file_path = '/content/dataset_procesado_FINAL_20250604_234332.csv'
df = pd.read_csv(file_path)

# Extraer la URL del texto usando regex
def extract_url(text):
    match = re.search(r'https?://\S+', str(text))
    return match.group(0) if match else None

df['url'] = df['texto_titular'].apply(extract_url)

# Renombrar columna y reordenar
df.rename(columns={'texto_titular': 'title'}, inplace=True)
df_filtered = df[['url', 'title', 'is_clickbait']]

# Guardar el primer dataset
df_filtered.to_csv('/content/dataset_con_urls.csv', index=False)
df_filtered.head()


,url,title,is_clickbait
0,https://t.co/ik70DfHNVe,"🤫 El secreto mejor guardado de Paula Badosa: ""...",1
1,https://t.co/QSJEl9GSSD,🎮 Aimar Bretos descubre cuándo se jubilan los ...,1
2,https://t.co/jggrhWLEP2,🔍 Esta es la causa de más del 80% de los cánce...,1
3,https://t.co/4cOcWdGsSE,Giro de 180º en los cajeros de toda la vida: e...,1
4,https://t.co/noMgLDNRaW,Usar las entradas y salidas con menos coste. h...,1


Ahora, preprocesamos el texto de los titulares para eliminar emojis, menciones, hashtags y demás. Con esto, obtendremos la parte de noticias clickbait del dataset final para clasificación.

In [ ]:
import pandas as pd
import re

# Cargar CSV
df = pd.read_csv("dataset_con_urls.csv")

# Renombrar la columna de URL si hace falta
if df.columns[0] != "url":
    df.rename(columns={df.columns[0]: "url"}, inplace=True)

# Función para limpiar texto
def limpiar_texto(texto):
    if pd.isna(texto):
        return ""

    # Eliminar emojis (Unicode extendido)
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002500-\U00002BEF"
        "\U00002702-\U000027B0"
        "\U00002700-\U000027BF"
        "\U000024C2-\U0001F251"
        "\U0001f926-\U0001f937"
        "\U00010000-\U0010ffff"
        "\u2640-\u2642"
        "\u2600-\u2B55"
        "\u200d"
        "\u23cf"
        "\u23e9"
        "\u231a"
        "\ufe0f"
        "\u3030"
        "]+", flags=re.UNICODE)
    texto = emoji_pattern.sub(r'', texto)

    # Eliminar menciones
    texto = re.sub(r'@\w+', '', texto)

    # Eliminar URLs
    texto = re.sub(r'http\S+|www.\S+', '', texto)

    # Eliminar hashtags
    texto = re.sub(r'#\w+', '', texto)

    # Eliminar barras verticales |
    texto = texto.replace('|', '')

    return texto.strip()

# Aplicar limpieza
df["title"] = df["title"].astype(str).apply(limpiar_texto)

# Eliminar filas con la palabra 'error'
df = df[~df["title"].str.contains("error", case=False)]

# Mantener solo las columnas deseadas y ordenarlas
df = df[["url", "title", "is_clickbait"]]

# Guardar archivo limpio
df.to_csv("dataset_clickbait_clasificacion_limpio.csv", index=False)

Aquí usamos JSON-LD para obtener el contenido de las noticias, que usaremos en el modelo QA.

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from tqdm import tqdm

# Cargar el dataset generado en el paso anterior
df = pd.read_csv('/content/dataset_clickbait_clasificacion_limpio.csv')

def extract_jsonld_content(url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Buscar todos los scripts de tipo application/ld+json
        scripts = soup.find_all('script', type='application/ld+json')
        for script in scripts:
            try:
                data = json.loads(script.string)

                # Si es una lista de objetos JSON-LD
                if isinstance(data, list):
                    for item in data:
                        if isinstance(item, dict) and 'articleBody' in item:
                            return item['articleBody']
                elif isinstance(data, dict) and 'articleBody' in data:
                    return data['articleBody']
            except (json.JSONDecodeError, TypeError):
                continue
    except Exception as e:
        return None

# Aplicar la función con barra de progreso
tqdm.pandas()
df['content'] = df['url'].progress_apply(extract_jsonld_content)

# Guardar el dataset final
df.to_csv('/content/dataset_clickbait_QA.csv', index=False)
df.head()

100%|██████████| 393/393 [04:47<00:00,  1.37it/s]


,url,title,is_clickbait,content
0,https://t.co/ik70DfHNVe,"El secreto mejor guardado de Paula Badosa: ""Lo...",1,Pese a la fiebre y al malestar con el que Paul...
1,https://t.co/QSJEl9GSSD,Aimar Bretos descubre cuándo se jubilan los ju...,1,La Cadena SER ha emitido este jueves una edici...
2,https://t.co/jggrhWLEP2,Esta es la causa de más del 80% de los cáncere...,1,None
3,https://t.co/4cOcWdGsSE,Giro de 180º en los cajeros de toda la vida: e...,1,None
4,https://t.co/noMgLDNRaW,Usar las entradas y salidas con menos coste.,1,None


Por último, preprocesamos el texto del contenido para obtener la parte de noticias clickbait del dataset final para QA.

In [ ]:
!pip install ftfy

import requests
from bs4 import BeautifulSoup
import json
from tqdm import tqdm
import pandas as pd
import re
import html
import ftfy  # Librería para reparar texto mal codificado

# Cargar el dataset anterior
df = pd.read_csv('/content/dataset_clickbait_QA.csv')

def extract_jsonld_content(url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Buscar todos los scripts de tipo application/ld+json
        scripts = soup.find_all('script', type='application/ld+json')
        for script in scripts:
            try:
                data = json.loads(script.string)

                # Si es una lista de objetos JSON-LD
                if isinstance(data, list):
                    for item in data:
                        if isinstance(item, dict) and 'articleBody' in item:
                            return item['articleBody']
                elif isinstance(data, dict) and 'articleBody' in data:
                    return data['articleBody']
            except (json.JSONDecodeError, TypeError):
                continue
    except Exception:
        return None

# Limpieza del texto
def clean_text(text):
    if pd.isna(text):
        return None

    # Reparar codificación (ej. Ã¡ → á)
    text = ftfy.fix_text(text)

    # Quitar HTML
    text = BeautifulSoup(text, "html.parser").get_text()

    # Decodificar entidades HTML (como &nbsp;)
    text = html.unescape(text)

    # Eliminar URLs
    text = re.sub(r'https?://\S+', '', text)

    # Eliminar menciones y hashtags
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)

    # Eliminar emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticonos
        u"\U0001F300-\U0001F5FF"  # pictogramas
        u"\U0001F680-\U0001F6FF"  # transporte/mapas
        u"\U0001F700-\U0001F77F"  # alquimia
        u"\U0001F780-\U0001F7FF"  # símbolos geométricos
        u"\U0001F800-\U0001F8FF"  # flechas
        u"\U0001F900-\U0001F9FF"  # emojis adicionales
        u"\U0001FA00-\U0001FA6F"  # instrumentos musicales
        u"\U0001FA70-\U0001FAFF"  # partes del cuerpo
        u"\u2600-\u26FF"          # símbolos misceláneos
        u"\u2700-\u27BF"          # Dingbats
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub('', text)

    # Eliminar frases genéricas de enlaces
    text = re.sub(r'(Vea también|Le puede interesar)[^\.]*\.', '', text, flags=re.IGNORECASE)

    # Eliminar espacios múltiples
    text = re.sub(r'\s+', ' ', text).strip()

    return text if text else None

# Aplicar la extracción
tqdm.pandas()
df['content'] = df['url'].progress_apply(extract_jsonld_content)

# Limpiar el texto
df['content'] = df['content'].apply(clean_text)

# Eliminar filas con contenido nulo o vacío
df = df.dropna(subset=['content'])

# Guardar el dataset limpio
df.to_csv('/content/dataset_clickbait_QA_limpio.csv', index=False)
df.head()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00


100%|██████████| 393/393 [05:18<00:00,  1.23it/s]


,url,title,is_clickbait,content
0,https://t.co/ik70DfHNVe,"El secreto mejor guardado de Paula Badosa: ""Lo...",1,Pese a la fiebre y al malestar con el que Paul...
1,https://t.co/QSJEl9GSSD,Aimar Bretos descubre cuándo se jubilan los ju...,1,La Cadena SER ha emitido este jueves una edici...
6,https://t.co/KiaFQWg0V1,Retiran estos populares licores vendidos en to...,1,La Agencia Española de Seguridad Alimentaria y...
7,https://t.co/o86UhKS0Nc,Este es el país con la costa más larga del mun...,1,Una de las consultas frecuentes en Google está...
8,https://t.co/Gvx8QYzHW0,El bonito pueblo milenario en el que nació Ait...,1,Aitana visita hoy El Hormiguero y protagoniza ...
